In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from os.path import join as oj
import sys
sys.path.append('../vision_fit')
sys.path.append('../vision_analyze')
import viz_weights
import numpy as np
from copy import deepcopy
import pickle as pkl
from torch.optim.lr_scheduler import StepLR
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise
import matplotlib.pyplot as plt
import models
from dim_reduction import *
from sklearn.decomposition import MiniBatchDictionaryLearning
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
# load dset
root = oj('/scratch/users/vision/yu_dl/raaz.rsk/data', 'cifar10')
trans = transforms.Compose([transforms.ToTensor()])
test_set = dset.CIFAR10(root=root, train=False, download=True)
X_test = test_set.test_data
Y_test = np.array(test_set.test_labels)
lab_dict = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

# filter by a class
idxs = Y_test==1
X = X_test[idxs]
Y = Y_test[idxs]

# look at an image or 2
# num = 0
# plt.imshow(X[num], interpolation=None)
# plt.title(lab_dict[Y[num]])
# plt.show()

X_d = X.reshape(X.shape[0], -1)
print(X_d.shape)

Files already downloaded and verified
(1000, 3072)


In [ ]:
# alpha = 1.0
# n_iter = 1
# n_components = 256
# batch_size = 100

# dico = MiniBatchDictionaryLearning(n_components=n_components, alpha=alpha, n_iter=n_iter, n_jobs=1, batch_size=batch_size) 
# for i in tqdm(range(50000)):
#     V = dico.fit(X_d)
#     if i % 100 == 0:
#         np.save('sparse_cifar/bases_iters=' + str(i) + '_alpha=' + str(alpha) + '_ncomps=' + str(n_components) + '.npy', V.components_)        
# #         viz_weights.plot_weights(V.components_, dset='cifar10')

In [ ]:
# viz_weights.plot_weights(V.components_, dset='rgb')

# get gpu code to work...

In [3]:
sys.path.append('ke_sparse_coding_pytorch/EE290T_quantized_sparse_codes')
from ke_sparse_coding_pytorch.EE290T_quantized_sparse_codes.training import sparse_coding

In [1]:
num_bases = 10
X_t = torch.Tensor(X.reshape(1000, -1, 1)).cuda()
bases_init = np.random.uniform(size=(1000, num_bases))
bases_init = torch.Tensor(bases_init)
d = {'sparsity_weight': 1, 'max_num_iters': 1}
sched = {x:d for x in range(4)}
sparse_coding.train_dictionary(X_t, bases_init, all_params={'num_epochs': 1, 'code_inference_algorithm': 'ista', 'dictionary_update_algorithm': 'sc_steepest_descent', 
                                                          'inference_param_schedule': sched, 'dict_update_param_schedule': sched})


NameError: name 'torch' is not defined

In [ ]:
"""
Train a sparse coding dictionary. These settings for Field natural images dset
"""
import sys
import os
# examples_fullpath = os.path.dirname(os.path.abspath(__file__))
# toplevel_dir_fullpath = examples_fullpath[:examples_fullpath.rfind('/')+1]
# sys.path.insert(0, toplevel_dir_fullpath)
sys.path.append('ke_sparse_coding_pytorch/EE290T_quantized_sparse_codes')

import argparse
import pickle
import numpy as np
from matplotlib import pyplot as plt
import torch

from training.sparse_coding import train_dictionary as sc_train
from utils.plotting import TrainingLivePlot
from utils.image_processing import create_patch_training_set


In [ ]:
image_patches_gpu = torch.Tensor(X_d[:10]).cuda()

In [ ]:
RUN_IDENTIFIER = 'test_sparse_coding'

BATCH_SIZE = 250
NUM_BATCHES = 4000  # 1 million patches total
PATCH_HEIGHT = 16
PATCH_WIDTH = 16

CODE_SIZE = 256
NUM_EPOCHS = 30

SC_PARAMS = {
    'num_epochs': NUM_EPOCHS,
    'code_inference_algorithm': 'fista',
    'inference_param_schedule': {
      0: {'sparsity_weight': 0.1, 'max_num_iters': 50},
      10*NUM_BATCHES: {'sparsity_weight': 0.1, 'max_num_iters': 100},
      20*NUM_BATCHES: {'sparsity_weight': 0.1, 'max_num_iters': 200}},
    'dictionary_update_algorithm': 'sc_cheap_quadratic_descent',
    'dict_update_param_schedule': {
      0: {'stepsize': 0.05, 'num_iters': 1},
      10*NUM_BATCHES: {'stepsize': 0.01, 'num_iters': 1},
      20*NUM_BATCHES: {'stepsize': 0.005, 'num_iters': 1}},
    'training_visualization_schedule': {0: None, 1000: None, 2000: None}}
SC_PARAMS['training_visualization_schedule'].update(
    {NUM_BATCHES*x: None for x in range(NUM_EPOCHS)})

# Arguments for dataset and logging
parser = argparse.ArgumentParser()
parser.add_argument("data_id",
    help="Name of the dataset (currently allowable: " +
         "Field_NW_whitened, Field_NW_unwhitened)")
parser.add_argument("data_filepath", help="The full path to dataset on disk")
parser.add_argument("-l", "--logfile_dir",
                    help="Optionally checkpoint the model here")
# script_args = parser.parse_args()

# if script_args.logfile_dir is not None:
SC_PARAMS['checkpoint_schedule'] = {'checkpoint_folder_fullpath': RUN_IDENTIFIER,
      NUM_BATCHES: None, 10*NUM_BATCHES: None, 20*NUM_BATCHES:None}

# torch_device = torch.device('cuda:1')
# torch.cuda.set_device(1)
# otherwise can put on 'cuda:0' or 'cpu'

# manually create large training set with one million whitened patches
'''
one_mil_image_patches = create_patch_training_set(
    ['patch'], (PATCH_HEIGHT, PATCH_WIDTH), BATCH_SIZE, NUM_BATCHES,
    edge_buffer=5, dataset=script_args.data_id,
    datasetparams={'filepath': script_args.data_filepath,
                   'exclude': []})['batched_patches']
'''

#################################################################
# save these to disk if you want always train on the same patches
# or if you want to speed things up in the future
#################################################################
# pickle.dump(one_mil_image_patches, open('/media/expansion1/spencerkent/Datasets/Field_natural_images/one_million_patches_October24.p', 'wb'))

# one_mil_image_patches = pickle.load(open(
#     '/media/expansion1/spencerkent/Datasets/Field_natural_images/one_million_patches_October24.p', 'rb')).astype('float32')

# send ALL image patches to the GPU
# image_patches_gpu = torch.from_numpy(one_mil_image_patches).to(torch_device)

# create the dictionary Tensor on the GPU
sparse_coding_dictionary = torch.randn((PATCH_HEIGHT*PATCH_WIDTH, CODE_SIZE))
sparse_coding_dictionary.div_(sparse_coding_dictionary.norm(p=2, dim=0))

# Create the LivePlot object
liveplot_obj = TrainingLivePlot(
    dict_plot_params={'total_num': CODE_SIZE, 'img_height': PATCH_HEIGHT,
                      'img_width': PATCH_WIDTH, 'plot_width': 16,
                      'plot_height': 16, 'renorm imgs': True,
                      'display_ordered': True},
    code_plot_params={'size': CODE_SIZE})

SC_PARAMS['training_visualization_schedule']['liveplot_object_reference'] = liveplot_obj

print("Here we go!")
sc_train(image_patches_gpu, sparse_coding_dictionary, SC_PARAMS)
